In [22]:
from pydataset import data
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing


from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

## Load the tips dataset.

### A. Create a column named tip_percentage. This should be the tip amount divided by the total bill.


In [2]:
tips = data('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
tips['tip_percentage'] = tips.tip / tips.total_bill

In [4]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


### B. Create a column named price_per_person. This should be the total bill divided by the party size.

In [7]:
# Note tips.size kept messing me up need tips['size']

tips['price_per_person'] = round(tips.total_bill/tips['size'], 2)

In [8]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.15


### C. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

size and total bill are the most realistic for predicting tip.
tip percentage would likely cause target leakage and not truly a predictor

### D. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

In [18]:
#from sklearn.feature_selection import SelectKBest, f_regression



X = tips[['total_bill', 'size', 'tip_percentage', 'price_per_person']]
#target
y = tips.tip

#Initialize - top two features
f_selector = SelectKBest(f_regression, k=2)
#fit the object - find top 2 X's correlated with y
f_selector.fit(X, y)
#boolean mask of whether the column was selected or not
f_mask = f_selector.get_support()
#get list of 2 K features
f_feature = X.iloc[:,f_mask].columns.tolist()

f_feature

['total_bill', 'size']

In [23]:
# from sklearn.linear_model import LinearRegression
# from sklearn.feature_selection import RFE

#initialize 
lm = LinearRegression()
# create the rfe object, indicating the ML object (lm) 
#and the number of features I want to end up with. 
rfe = RFE(lm, 2)
# fit the data using RFE
rfe.fit(X,y)
# get the mask of the columns selected
f_mask = rfe.support_
# get list of the column names. 
rfe_feature = X.iloc[:,f_mask].columns.tolist()

rfe_feature

['total_bill', 'tip_percentage']

### E. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?

In [24]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.15


In [30]:
# K best and recursive for tip percentage

#should be the same code just change x and y

X = tips[['total_bill', 'size','price_per_person']]
#target
y = tips['tip_percentage']

# did not add string columns

In [33]:
#Initialize - top two features
f_selector = SelectKBest(f_regression, k=2)
#fit the object - find top 2 X's correlated with y
f_selector.fit(X, y)
#boolean mask of whether the column was selected or not
f_mask = f_selector.get_support()
#get list of 2 K features
f_feature = X.iloc[:,f_mask].columns.tolist()

f_feature

['total_bill', 'price_per_person']

In [34]:
#initialize 
lm = LinearRegression()
# create the rfe object, indicating the ML object (lm) 
#and the number of features I want to end up with. 
rfe = RFE(lm, 2)
# fit the data using RFE
rfe.fit(X,y)
# get the mask of the columns selected
f_mask = rfe.support_
# get list of the column names. 
rfe_feature = X.iloc[:,f_mask].columns.tolist()

rfe_feature

['size', 'price_per_person']

### F. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

K Best is based on statistical testing 

Recursive feature elimination is based on modeling 

as you change some of the features that should be some similarties given this particular dataset

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually

In [35]:
#predictors (X), target (y), n_features
def select_kbest(predictors, target, n_features):
    f_selector = SelectKBest(f_regression, k= n_features)
    f_selector.fit(predictors, target)
    f_mask = f_selector.get_support()
    f_feature = predictors.iloc[:,f_mask].columns.tolist()
    return f_feature

In [37]:
#test
select_kbest(tips[['total_bill', 'size', 'price_per_person']], tips.tip, 2)

['total_bill', 'size']

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [39]:
def rfe(predictors, target, n_features):

    lm = LinearRegression()
    rfe = RFE(lm, n_features)
    rfe.fit(predictors, target)
    f_mask = rfe.support_
    rfe_feature = predictors.iloc[:,f_mask].columns.tolist()

    return rfe_feature

In [40]:
#test
rfe(tips[['total_bill', 'size', 'price_per_person']], tips.tip, 2)

['total_bill', 'price_per_person']

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [44]:
swiss = data('swiss')

swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [45]:
swiss.shape

(47, 6)

In [ ]:
select_kbest(swiss.drop(columns='Fertility'), swiss['Fertility'])